IMPORT All THE NEEDED LIBRARIES


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import pickle

Next step import your data

In [ ]:
Data = pd.read_csv('survey lung cancer.csv')
Data.head(10)

In [ ]:
#Now that we have imported the data we need to understand some things
#*1 what is our dataset saying to us 
#*2 what are the  problems of our data
#*3 what are the things not need in my data
#*4 which algorithm should i use is it supervised or unsupervised
#*5 if supervised which algorithm is the best to solve my problem

Visualize you Dataset

In [ ]:
sns.histplot(Data['AGE'].head())

In [ ]:
import matplotlib.pyplot as plt
Age = Data['AGE']
peerpressure = Data['PEER_PRESSURE']
smoking = Data['SMOKING']
plt.pie(smoking.head())


In [ ]:
plt.bar(Data['GENDER'].head(),Age.head())

In [ ]:
plt.bar(Data['GENDER'].head(),smoking.head())

In [ ]:
Data.info()

In [ ]:
Data.count()

In [ ]:
Data.shape

In [ ]:
Data.describe()

In [ ]:
Data.isnull().values.any()

In [ ]:
def plot_corr(Data, size=11):
    

    corr = Data.corr()    # data frame correlation function
    fig, ax = plt.subplots(figsize=(size, size))
    ax.matshow(corr)   # color code the rectangles by correlation value
    plt.xticks(range(len(corr.columns)), corr.columns)  # draw x tick marks
    plt.yticks(range(len(corr.columns)), corr.columns) 

In [ ]:
del Data['CHEST PAIN']

In [ ]:
plot_corr(Data)

In [ ]:
Data.corr()

In [ ]:
Data.head()

In [ ]:
del Data['FATIGUE ']

In [ ]:
del Data['YELLOW_FINGERS']

In [ ]:
plot_corr(Data)

In [ ]:
Data.isnull().values.any()

In [ ]:
Data.loc[Data['LUNG_CANCER'] == 'NO','LUNG_CANCER' ] = 0
Data.loc[Data['LUNG_CANCER'] == 'YES','LUNG_CANCER' ] = 1

In [ ]:
Data.loc[Data['GENDER'] == 'M','GENDER' ] = 0
Data.loc[Data['GENDER'] == 'F','GENDER' ] = 1

In [ ]:
Data.head()

In [ ]:
num_obs = len(Data)
num_true = len(Data.loc[Data['LUNG_CANCER'] == 1])
num_false = len(Data.loc[Data['LUNG_CANCER'] == 0])
print("Number of True cases:  {0} ({1:2.2f}%)".format(num_true, (num_true/num_obs) * 100))
print("Number of False cases: {0} ({1:2.2f}%)".format(num_false, (num_false/num_obs) * 100))

In [ ]:

from sklearn.model_selection import train_test_split
feature_col_names = Data.columns
predicted_class_names = ['LUNG_CANCER']

X =  Data[feature_col_names] 
X = X.astype('int')# predictor feature columns (8 X m)
y = Data[predicted_class_names] # predicted class (1=true, 0=false) column (1 X m)
y = y.astype('int')
lebel_en2 = LabelEncoder()
split_test_size = 0.30
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=split_test_size,shuffle= True, 
                                                      random_state=0) 
y_test = lebel_en2.fit_transform(y_test)

In [ ]:
print("{0:0.2f}% in training set".format((len(X_train)/len(Data.index)) * 100))
print("{0:0.2f}% in test set".format((len(X_test)/len(Data.index)) * 100))

In [ ]:
from sklearn.impute import SimpleImputer

#Impute with mean all 0 readings
fill_0 = SimpleImputer(missing_values=0, strategy="mean")

# Notice the missing_values=0 will be replaced by mean.  However, the num_preg can have a value of 0.
# To prevent replacing the 0 num_preg with the mean we need to skip imputing the 'num_preg' column
cols_not_num_preg = X_train.columns.difference(['num_preg']) # all columns but the num_preg column
pd.options.mode.chained_assignment = None  # Supress warning message on transformed assignment

# impute the training data
X_train[cols_not_num_preg] = fill_0.fit_transform(X_train[cols_not_num_preg])

# impute the test data
X_test[cols_not_num_preg] = fill_0.transform(X_test[cols_not_num_preg])

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier(criterion='entropy', max_depth=2)      # Create random forest object
rf_model.fit(X_train, y_train.values.flatten()) 

### Predict Training Data

In [ ]:
rf_predict_train = rf_model.predict(X_train)
# training metrics
print("Accuracy: {0:.4f}".format(metrics.accuracy_score(y_train, rf_predict_train)))

### Predict Test Data

In [ ]:
rf_predict_test = rf_model.predict(X_test)

# training metrics
print("Accuracy: {0:.4f}".format(metrics.accuracy_score(y_test, rf_predict_test)))

In [ ]:
sns.histplot(rf_predict_test)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
# Model accuracy
plt.hist(rf_predict_train)
plt.hist(rf_predict_test)
plt.title('Model Accuracy')
plt.ylabel('accuracy')
plt.xlabel('n_estimators')
plt.legend(['train', 'test'])
plt.show()

In [ ]:
filename = 'rf_model.sav'
pickle.dump(rf_model, open(filename, 'wb'))